Importing libraries

In [1]:
import pandas as pd
from google.cloud import vision, storage
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\Additional-Task\\storage_key.json"

Reading csv file containing product_name and url

In [3]:
data = pd.read_csv("D:\\Work and Assignments\\Python\\sampleCSV.csv")
data.shape

(99, 3)

In [4]:
s_data = data[['product_name', 'product_url']]
s_data

,product_name,product_url
0,Top,https://i.pinimg.com/originals/2e/88/a5/2e88a5...
1,Watch,https://w7.pngwing.com/pngs/566/206/png-transp...
2,Shirt,https://funkybabaji.com/cdn/shop/files/9.png?v...
3,Shoe,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,Top,https://i.pinimg.com/originals/2e/88/a5/2e88a5...
...,...,...
94,tv,https://www.jansatta.com/wp-content/uploads/20...
95,Top,https://i.pinimg.com/originals/2e/88/a5/2e88a5...
96,Watch,https://w7.pngwing.com/pngs/566/206/png-transp...
97,Shirt,https://funkybabaji.com/cdn/shop/files/9.png?v...


Storing url column in data_url

In [5]:
data_url = data['product_url']
data_url

0     https://i.pinimg.com/originals/2e/88/a5/2e88a5...
1     https://w7.pngwing.com/pngs/566/206/png-transp...
2     https://funkybabaji.com/cdn/shop/files/9.png?v...
3     https://encrypted-tbn0.gstatic.com/images?q=tb...
4     https://i.pinimg.com/originals/2e/88/a5/2e88a5...
                            ...                        
94    https://www.jansatta.com/wp-content/uploads/20...
95    https://i.pinimg.com/originals/2e/88/a5/2e88a5...
96    https://w7.pngwing.com/pngs/566/206/png-transp...
97    https://funkybabaji.com/cdn/shop/files/9.png?v...
98    https://encrypted-tbn0.gstatic.com/images?q=tb...
Name: product_url, Length: 99, dtype: object

Function to check if url contains a image or not

In [37]:
# import requests
# def is_url_image(url):
#    results = []
#    response = requests.head(url, timeout=5)  # Use HEAD request to check headers only
#    content_type = response.headers.get('Content-Type')
#    if content_type and 'image' in content_type:
#       results.append((url, True))
#    else:
#       results.append((url, False))

#    return pd.DataFrame(results, columns=['URL', 'IsImage'])

In [5]:
# import webcolors
# def closest_colour(requested_colour):
#     min_colours = {}
#     for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
#         r_c, g_c, b_c = webcolors.hex_to_rgb(key)
#         rd = (r_c - requested_colour[0]) ** 2
#         gd = (g_c - requested_colour[1]) ** 2
#         bd = (b_c - requested_colour[2]) ** 2
#         min_colours[(rd + gd + bd)] = name
#     return min_colours[min(min_colours.keys())]

# def get_colour_name(requested_colour):
#     try:
#         closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
#     except ValueError:
#         closest_name = closest_colour(requested_colour)
#         actual_name = None
#     return actual_name, closest_name

# requested_colour = (119, 172, 152)
# actual_name, closest_name = get_colour_name(requested_colour)

In [7]:

# def detect_properties_uri(uri):
#     client = vision.ImageAnnotatorClient()
#     image = vision.Image()
#     image.source.image_uri = uri

#     response = client.image_properties(image=image)
#     props = response.image_properties_annotation
#     # print("Properties:",props.dominant_colors.colors)
#     # scr = [col for col in props.dominant_colors.colors]
#     # print(scr)
#     for color in props.dominant_colors.colors:
#         try:
#             r = int(color.color.red)
#             g = int(color.color.green)
#             b = int(color.color.blue)
#             print(f"color: {get_colour_name((r,g,b))}")
#         except Exception as e:
#             print("gone from here",e)
#             continue
        
# detect_properties_uri(data_url[1])

color: (None, 'black')
color: (None, 'darkslategray')
color: (None, 'rosybrown')
color: (None, 'whitesmoke')
color: (None, 'darkslategray')
color: (None, 'black')
color: (None, 'darkslategray')
color: (None, 'gray')
color: (None, 'black')
color: (None, 'darkslategray')


Empty String variable to store results

In [5]:
txt = ""

Function to extract information from image

In [4]:
def extract_info(uri):
    txt = ''
    l = ''
    try:
        # if(is_url_image(uri)==True):
            client = vision.ImageAnnotatorClient()
            img = vision.Image()
            img.source.image_uri = uri
            
            #For retriving Brand
            # res_logo = client.logo_detection(img)
            # logos = res_logo.logo_annotations
            # logo = [log.description for log in logos]
            # txt = txt + f"Brand: None, " if len(logo)==0 else txt + f"Brand: {logo[0]}, "
            
            #For retriving Labels
            res_label = client.label_detection(img)
            labels = res_label.label_annotations
            label = [lab.description for lab in labels]
            # txt = txt + f"Description: None, " if len(label)==0 else txt + f"Description: {str(label)}, "
            l += f"{label}"
            #For retriving Text
            # res_text = client.document_text_detection(img)
            # texts = res_text.text_annotations
            # text = [t.description for t in texts]
            # txt = txt + f"Text: None, " if len(text)==0 else txt + f"Text: {text[0]}, "
            
            objects = client.object_localization(image=img).localized_object_annotations
            obj = [ob.name for ob in objects]
            txt = txt = list(set(obj).intersection(label)) if set(obj).intersection(label) else label[:2] if len(obj) == 0 else set(obj)
        
        # else:
        #     txt = "Please enter a valid URL that contains Image"
    except Exception as e:
        txt = f"Please Enter a Valid URL of Image:{e}"
    return txt,l   

Creating new dataframe to save results

In [7]:
result = s_data
result.head()

,product_name,product_url
0,Top,https://i.pinimg.com/originals/2e/88/a5/2e88a5...
1,Watch,https://w7.pngwing.com/pngs/566/206/png-transp...
2,Shirt,https://funkybabaji.com/cdn/shop/files/9.png?v...
3,Shoe,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,Top,https://i.pinimg.com/originals/2e/88/a5/2e88a5...


Iterating through function to extract features of image and adding it to result dataframe

In [8]:
for i in range(len(data_url)):
    x=''
    y=''
    x,y = extract_info(data_url[i])
    result.loc[i, ["object", "label"]] = str(x), str(y)  

Resultant Dataframe

In [9]:
result

,product_name,product_url,object,label
0,Top,https://i.pinimg.com/originals/2e/88/a5/2e88a5...,{'Top'},"['Outerwear', 'Neck', 'Sleeve', 'T-shirt', 'Gr..."
1,Watch,https://w7.pngwing.com/pngs/566/206/png-transp...,['Watch'],"['Watch', 'Analog watch', 'Photograph', 'White..."
2,Shirt,https://funkybabaji.com/cdn/shop/files/9.png?v...,"{'Top', 'Shirt'}","['Sleeve', 'Collar', 'Magenta', 'Button', 'Pat..."
3,Shoe,https://encrypted-tbn0.gstatic.com/images?q=tb...,['Shoe'],"['Footwear', 'Shoe', 'Walking shoe', 'Cleat', ..."
4,Top,https://i.pinimg.com/originals/2e/88/a5/2e88a5...,{'Top'},"['Outerwear', 'Neck', 'Sleeve', 'T-shirt', 'Gr..."
...,...,...,...,...
94,tv,https://www.jansatta.com/wp-content/uploads/20...,{'Television'},"['Flower', 'Plant', 'Petal', 'Font', 'Rectangl..."
95,Top,https://i.pinimg.com/originals/2e/88/a5/2e88a5...,{'Top'},"['Outerwear', 'Neck', 'Sleeve', 'T-shirt', 'Gr..."
96,Watch,https://w7.pngwing.com/pngs/566/206/png-transp...,['Watch'],"['Watch', 'Analog watch', 'Photograph', 'White..."
97,Shirt,https://funkybabaji.com/cdn/shop/files/9.png?v...,"{'Top', 'Shirt'}","['Sleeve', 'Collar', 'Magenta', 'Button', 'Pat..."


Saving Dataframe to CSV file

In [10]:
result.to_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv",index=False)